# Прогнозирование выполнения задач фазы 5 методом Монте-Карло

Смотря на историю выполнения предыдущих, сделайте прогноз выполнения задач 5 фазы методом Монте-Карло, моделируя двумя путями:
1. Используйте статистику количества задач, выполненных во все прошедшие недели. Рассматривайте это как дискретное распределение случайной величины. Прогоните простенькую модель симуляции выполнения задач 5 фазы (на каждой итерации сэмплируем число выполненных за неделю задач из дискретного распределения). Оцените с 80% достоверностью срок выполнения 5 фазы (в неделях).
2. Опишите случайной величиной суммарные трудозатраты, которые вашей команде удаётся выделять данному предмету в неделю. Тоже возьмите для этого предыдущую статистику (сколько в какую неделю удавалось уделить человеко часов) и рассматривайте как дискретное распределение. В симуляции на каждом запуске продолжительности задач сэмплируйте из треугольного распределения (с параметрами, соответствующими трёхточечной оценке). Оцените с 80% достоверностью срок выполнения 5 фазы в соответствие с этой моделью (в неделях).

In [74]:
from datetime import date

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [75]:
data = pd.read_excel('drive/MyDrive/education/PM_practics/Yougile_stats.xlsx')

def str_to_date(s):
  d, m, y = s.split(' ')[0].split('.')
  return date(int(y), int(m), int(d))

def week_no(d: date):
  return d.isocalendar().week

data = data.iloc[:31] # убираем ИТОГО
data['Work, h.'] = data['Work, h.'].apply(float)
data['Completed at'] = data['Completed at'].apply(str_to_date) # приводим строковые данные к формату даты
data['week_no'] = data['Completed at'].apply(week_no) # добавляем поле с номером недели года, когда была выполнена задача
data

/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,"Work, h.",ID Task,Task title,Created at,Author,Assigned to / Manager,Deadline,Completed at,week_no
0,1.50000,ITP-34,Встреча по планированию фазы 4 и 5,18.10.2025 12:15,v.gorb-123456789@yandex.ru,"Боев Максим, Валерия Самарченко, v.gorb-123456...",NaN,2025-10-18,42
1,0.16667,ITP-32,Сдать Фазу 2,07.10.2025 17:49,Валерия Самарченко,"Валерия Самарченко, v.gorb-123456789@yandex.ru...",15.10.2025 22:59,2025-10-15,42
2,0.33333,ITP-33,Сдать Фазу 3,14.10.2025 20:33,Валерия Самарченко,NaN,15.10.2025 22:59,2025-10-15,42
3,1.00000,ITP-26,Решить сколько сотрудников + какие должности н...,05.10.2025 16:55,Валерия Самарченко,Валерия Самарченко,NaN,2025-10-14,42
4,1.05000,ITP-28,Какие фичи первостепенны,05.10.2025 16:56,Валерия Самарченко,"Валерия Самарченко, Боев Максим, v.gorb-123456...",NaN,2025-10-14,42
5,1.00000,ITP-20,Сделать часть 1 с use cases,05.10.2025 16:53,Валерия Самарченко,Боев Максим,NaN,2025-10-14,42
6,1.33333,ITP-21,Сделать часть 2 с use cases,05.10.2025 16:53,Валерия Самарченко,v.gorb-123456789@yandex.ru,NaN,2025-10-14,42
7,1.25000,ITP-22,Сделать часть 3 с use cases,05.10.2025 16:53,Валерия Самарченко,Валерия Самарченко,NaN,2025-10-14,42
8,1.50000,ITP-25,Оценить проект (Покер/коллективно),05.10.2025 16:54,Валерия Самарченко,"Валерия Самарченко, v.gorb-123456789@yandex.ru...",NaN,2025-10-14,42
9,3.00000,ITP-18,Создать не менее 12 use cases,05.10.2025 16:52,Валерия Самарченко,v.gorb-123456789@yandex.ru,14.10.2025 22:59,2025-10-14,42


In [76]:
# получаем распределение количества задач, выполненых в предыдущие недели
task_per_week = data.groupby(by='week_no').count()['ID Task'].to_numpy()
task_per_week

array([ 8,  8,  2, 13])

In [77]:
# получаем трудозатраты за предыдущие недели
hhour_per_week = data[['week_no', 'Work, h.']].groupby(by='week_no').sum().to_numpy().ravel()
hhour_per_week

array([14.33334, 12.08334,  6.     , 16.46666])

In [78]:
# метод Монте-Карло на основе недельной производительности (задач/неделя)
iter_count = 10000
tasks_count = 5

weeks_result_mc1 = []

for i in range(iter_count):
  completed_tasks = 0
  weeks = 0
  while completed_tasks < tasks_count:
    weeks += 1
    completed_tasks += np.random.choice(task_per_week)

  weeks_result_mc1.append(weeks)

In [79]:
weeks_result_mc1 = np.array(weeks_result_mc1)
print(f'С 80-процентной достоверностью, задачи будут выполнены за (недель): {np.percentile(weeks_result_mc1, 80)}')

С 80-процентной достоверностью, задачи будут выполнены за (недель): 2.0


In [80]:
# метод Монте-Карло на основе недельных трудозатрат и трёхточечной оценки текущих задач
iter_count = 10000
tasks_triangulars = [[1, 2, 3],
                    [0.5, 1, 3],
                    [0.5, 1, 2],
                    [2, 3, 4],
                    [2.5, 4, 6]]

weeks_result_mc2 = []

for i in range(iter_count):
  total_lc = np.sum([np.random.triangular(*tt) for tt in tasks_triangulars])
  completed_lc = 0
  weeks = 0
  while completed_lc < total_lc:
    weeks += 1
    completed_lc += np.random.choice(hhour_per_week)

  weeks_result_mc2.append(weeks)

In [83]:
weeks_result_mc2 = np.array(weeks_result_mc2)
print(f'С 80-процентной достоверностью, задачи будут выполнены за (недель): {np.percentile(weeks_result_mc2, 80)}')

С 80-процентной достоверностью, задачи будут выполнены за (недель): 2.0


Оценки сошлись - мы вполне корректно оценили наши задачи :)